# 법령 데이터 추출

## 1. install library

In [11]:
# !pip install --upgrade pip
# !pip install pymupdf
# !pip install sentence-transformers
# !pip install python-dotenv
# !pip install openai
# !pip install faiss-cpu
# !pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.9/907.9 kB 8.8 MB/s eta 0:00:00


## 2. 청크 테스트

### 1) 파일 리스트 추출

In [10]:
import os
print("경로 : "+os.getcwd())

def get_pdf_files(directory):
    # 해당 디렉토리에서 파일 목록을 가져와서 .pdf 확장자 파일만 필터링
    return [f for f in os.listdir(directory) if f.endswith('.pdf')]

# labor_law 디렉토리 안의 PDF 파일 목록 가져오기
pdf_directory = "labor_law" 
pdf_files = get_pdf_files(pdf_directory)

# PDF 파일 목록 출력
print("개수 : "+str(len(pdf_files)))
print(pdf_files)

경로 : /Users/henry/github/ai
개수 : 37
['산업현장 일학습병행 지원에 관한 법률.pdf', '가사근로자의 고용개선 등에 관한 법률.pdf', '공무원연금법.pdf', '고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률.pdf', '파견근로자 보호 등에 관한 법률.pdf', '구직자 취업촉진 및 생활안정지원에 관한 법률.pdf', '산업재해보상보험법.pdf', '근로기준법.pdf', '임금채권보장법.pdf', '기간제 및 단시간근로자 보호 등에 관한 법률.pdf', '채용절차의 공정화에 관한 법률.pdf', '외국인근로자의 고용 등에 관한 법률.pdf', '장애인고용촉진 및 직업재활법.pdf', '노동위원회법.pdf', '국가인권위원회법.pdf', '어선원 및 어선 재해보상보험법.pdf', '공무원의 노동조합 설립 및 운영 등에 관한 법률.pdf', '중대재해 처벌 등에 관한 법률.pdf', '근로자참여 및 협력증진에 관한 법률.pdf', '고용보험법.pdf', '고용정책 기본법.pdf', '최저임금법.pdf', '산업안전보건법.pdf', '노동조합 및 노동관계조정법.pdf', '근로자퇴직급여 보장법.pdf', '사회적기업 육성법.pdf', '근로복지기본법.pdf', '국민 평생 직업능력 개발법.pdf', '숙련기술장려법.pdf', '진폐의 예방과 진폐근로자의 보호 등에 관한 법률.pdf', '건설근로자의 고용개선 등에 관한 법률.pdf', '남녀고용평등과 일_가정 양립 지원에 관한 법률.pdf', '선원법.pdf', '직업안정법.pdf', '교원의 노동조합 설립 및 운영 등에 관한 법률.pdf', '고용상 연령차별금지 및 고령자고용촉진에 관한 법률.pdf', '공무원 재해보상법.pdf']


### 2) PyMuPDF를 이용한 text 추출

In [24]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""

    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()

    doc.close()
    return text

# for f in pdf_files:
#     pdf_path = "labor_law/"+f
#     print(pdf_path, end=" ")

pdf_path = "labor_law/산업현장 일학습병행 지원에 관한 법률.pdf"
law_text = extract_text_from_pdf(pdf_path)
print(law_text)

법제처                                                            1                                                       국가법령정보센터
산업현장 일학습병행 지원에 관한 법률
산업현장 일학습병행 지원에 관한 법률 ( 약칭: 일학습병행법 )
[시행 2022. 2. 18.] [법률 제18425호, 2021. 8. 17., 타법개정]
 
       제1장 총칙
             제1조(목적)
             제2조(기본이념)
             제3조(정의)
             제4조(국가 등의 책무)
             제5조(다른 법률과의 관계)
       제2장 일학습병행 추진 체계
             제6조(일학습병행 추진계획의 수립)
             제7조(일학습병행에 관한 중요 사항의 심의)
             제8조(일학습병행의 지역별 협력)
             제9조(학습기업과 구직자의 연계체계 구축)
             제10조(일학습병행 직종 및 교육훈련기준에 대한 조사ㆍ연구)
       제3장 일학습병행 및 교육훈련의 실시
             제11조(일학습병행 직종 및 교육훈련기준)
             제12조(시범사업)
             제13조(학습기업의 지정)
             제14조(학습기업의 지정 취소 등)
             제15조(일학습병행과정 개발 및 인정 등)
             제16조(일학습병행 공동훈련센터의 지정)
             제17조(사업장 외 교육훈련의 장소)
             제18조(일학습병행 공동훈련센터의 지정 취소 등)
             제19조(기업현장교사의 지정ㆍ운영)
             제20조(기업현장교사의 육성)
             제21조(학습근로계약의 체결)
             제22조(학습근로계약의 

### 3) 목차 추출

In [28]:
import re

def extract_table_of_contents(text):
    # '제n조'와 괄호 안의 내용을 추출하는 정규 표현식
    toc_pattern = r'(제\d+조\(.+?\))'
    toc_items = []

    # 모든 항목을 순차적으로 추출
    for item in re.findall(toc_pattern, text):
        # 이미 toc_items에 없다면 추가
        if item not in toc_items:
            toc_items.append(item)

    return toc_items

def extract_table_of_chapters(text):
    # '제n조'와 괄호 안의 내용을 추출하는 정규 표현식
    chapter_pattern = r'(제\d+장 .+?)\n'
    chapter_items = []

    # 모든 항목을 순차적으로 추출
    for item in re.findall(chapter_pattern, text):
        # 이미 toc_items에 없다면 추가
        if item not in chapter_items:
            chapter_items.append(item)

    return chapter_items

# 목차 추출
toc = extract_table_of_contents(law_text)
chapter = extract_table_of_chapters(law_text)

# 정렬된 목차 출력
print(chapter)
print(toc)

['제1장 총칙', '제2장 일학습병행 추진 체계', '제3장 일학습병행 및 교육훈련의 실시', '제4장 학습기업 및 학습근로자의 의무', '제5장 평가 및 자격', '제6장 보칙', '제7장 벌칙']
['제1조(목적)', '제2조(기본이념)', '제3조(정의)', '제4조(국가 등의 책무)', '제5조(다른 법률과의 관계)', '제6조(일학습병행 추진계획의 수립)', '제7조(일학습병행에 관한 중요 사항의 심의)', '제8조(일학습병행의 지역별 협력)', '제9조(학습기업과 구직자의 연계체계 구축)', '제10조(일학습병행 직종 및 교육훈련기준에 대한 조사ㆍ연구)', '제11조(일학습병행 직종 및 교육훈련기준)', '제12조(시범사업)', '제13조(학습기업의 지정)', '제14조(학습기업의 지정 취소 등)', '제15조(일학습병행과정 개발 및 인정 등)', '제16조(일학습병행 공동훈련센터의 지정)', '제17조(사업장 외 교육훈련의 장소)', '제18조(일학습병행 공동훈련센터의 지정 취소 등)', '제19조(기업현장교사의 지정ㆍ운영)', '제20조(기업현장교사의 육성)', '제21조(학습근로계약의 체결)', '제22조(학습근로계약의 종료)', '제23조(학습근로계약의 해지 제한)', '제24조(학습근로자의 계속고용)', '제25조(학습기업 사업주의 준수사항)', '제26조(학습근로자의 준수사항)', '제27조(학습근로시간 및 휴식)', '제28조(차별적 처우의 금지)', '제29조(이수증명서 발급 등)', '제30조(학습근로자에 대한 평가)', '제31조(일학습병행자격 취득 및 자격증 발급)', '제32조(일학습병행자격증의 대여 등 금지)', '제33조(일학습병행자격의 취소 등)', '제34조(지원)', '제35조(서류의 보존)', '제36조(지도ㆍ점검 등)', '제37조(수수료)', '제38조(청문)', '제39조(권한 등의 위임ㆍ위탁)', '제40조(벌칙)', '제41조(양벌규정)', '제42조(과태료)', '제16조(제1호 및 제4호는 제외한다)']

### 4) 목차에 따라 청크화

In [57]:
import re


def find_body_start(text, chapter_items):
    """
    본문 시작점을 찾는 함수
    """
    first_chapter_item = chapter_items[0]
    matches = list(re.finditer(re.escape(first_chapter_item), text))

    # 두 번째로 등장하는 위치를 반환
    if len(matches) >= 2:
        return matches[1].start()  # 두 번째 등장 위치
    return -1  # 없으면 -1 반환

def chunk_by_chapter(text, law_name, chapter_items, toc_items):
    """
    목차 항목을 기준으로 청크화하는 함수
    """
    # 본문 시작점을 찾음
    body_start_index = find_body_start(text, chapter_items)

    # 본문 시작점 이후의 텍스트만 청크화
    if body_start_index != -1:
        text = text[body_start_index:]

    # 목차의 각 장을 기준으로 본문을 분리
    chapter_pattern = '|'.join(re.escape(item) for item in chapter_items)
    chapters = re.split(f'({chapter_pattern})', text)
    
    chunked_texts = []
    current_chapter = "Unknown Chapter"
    
    for i in range(1, len(chapters), 2):
        current_chapter = chapters[i].strip()  # 장 정보
        chapter_body = chapters[i + 1]  # 장의 본문

        # 조항을 기준으로 청크화
        chunk_pattern = '|'.join(re.escape(item) for item in toc_items)
        chunks = re.split(f'({chunk_pattern})', chapter_body)

        # 청크별로 처리
        for j in range(1, len(chunks) - 1, 2):
            chunked_texts.append({
                "Law": law_name,
                "Chapter": current_chapter,
                "Title": chunks[j].strip(),
                "Body": chunks[j + 1].strip()
            })

    return chunked_texts
law_name = "산업현장 일학습병행 지원에 관한 법률"
# 청크화 작업
chunked_text = chunk_by_chapter(law_text, law_name, chapter, toc)

# 청크 결과 출력
for chunk in chunked_text:
    print(f"Law: {chunk['Law']}")
    print(f"Chapter: {chunk['Chapter']}")
    print(f"Title: {chunk['Title']}")
    print(f"Body: {chunk['Body']}")
    print("="*50)

Law: 산업현장 일학습병행 지원에 관한 법률
Chapter: 제1장 총칙
Title: 제1조(목적)
Body: 이 법은 공무원의 공무로 인한 부상ㆍ질병ㆍ장해ㆍ사망에 대하여 적합한 보상을 하고, 공무상 재해를 입은
공무원의 재활 및 직무복귀를 지원하며, 재해예방을 위한 사업을 시행함으로써 공무원이 직무에 전념할 수 있는 여
건을 조성하고, 공무원 및 그 유족의 복지 향상에 이바지함을 목적으로 한다.
Law: 산업현장 일학습병행 지원에 관한 법률
Chapter: 제1장 총칙
Title: 제2조(주관)
Body: 이 법에 따른 공무원 재해보상제도의 운영에 관한 사항은 인사혁신처장이 주관한다.
Law: 산업현장 일학습병행 지원에 관한 법률
Chapter: 제1장 총칙
Title: 제3조(정의)
Body: ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2024. 3. 19.>
1. “공무원”이란 공무에 종사하는 다음 각 목의 어느 하나에 해당하는 사람을 말한다.
가. 「국가공무원법」, 「지방공무원법」, 그 밖의 법률에 따른 공무원. 다만, 군인과 선거에 의하여 취임하는 공무원
은 제외한다.
나. 그 밖에 국가기관이나 지방자치단체에 근무하는 직원 중 대통령령으로 정하는 사람
2. “공무수행사망자”란 제1호 외의 사람으로서 사망 당시(부상 또는 질병으로 사망한 경우에는 그 부상 또는 질병
발생 당시를 말한다) 다음 각 목의 요건 모두에 해당하는 것으로 제6조에 따른 공무원재해보상심의회 심의를 거
쳐 인사혁신처장이 인정하는 사람을 말한다. 다만, 군인과 선거에 의하여 취임하는 공무원은 제외한다.
가. 국가 또는 지방자치단체의 사무를 수행할 것
나. 국가 또는 지방자치단체가 업무상 관리ㆍ감독 권한을 직접 또는 간접적으로 가지고 있을 것
다. 국가 또는 지방자치단체가 법령 또는 계약 등에 따라 보수 또는 수당 등을 직접 지급하거나 대통령령으로 정
하는 바에 따라 간접적으로 지급하고 있을 것
라. 「산업재해보상보험법」 또는 그 밖의 법령(이하 “「산업재해보상보험법」등

In [50]:
def save_law_chunks_to_text(law_name, chunks):
    # 파일 경로 설정
    text_directory = "labor_law_texts"
    if not os.path.exists(text_directory):
        os.makedirs(text_directory)

    file_path = os.path.join(text_directory, f"{law_name}.txt")
    print(file_path)
    # 파일 쓰기 모드로 열기
    with open(file_path, 'w', encoding='utf-8') as f:
        # 파일의 첫 줄에 구분자와 법률명 추가
        f.write(f"========= {law_name} =========\n\n")

        # 청크 데이터를 파일에 작성
        for chunk in chunks:
            f.write(f"Chapter: {chunk['Chapter']}\n")
            f.write(f"Title: {chunk['Title']}\n")
            f.write(f"Body: {chunk['Body']}\n")
            f.write("="*50 + "\n\n")  # 구분선 추가

def find_body_start_by_toc(text, toc_items):
    """
    목차 항목(toc_items) 중 첫 번째 항목이 두 번째로 등장하는 위치를 찾는 함수
    """
    if not toc_items:
        return -1  # 목차 정보가 없을 경우

    first_toc_item = toc_items[0]
    matches = list(re.finditer(re.escape(first_toc_item), text))

    # 두 번째로 등장하는 위치를 반환
    if len(matches) >= 2:
        return matches[1].start()  # 두 번째 등장 위치
    return -1  # 없으면 -1 반환

def chunk_by_toc(text, law_name, toc_items):
    """
    조항(toc)을 기준으로 청크화하는 함수
    """
    # 본문 시작점을 목차에서 찾음 (조를 기준으로)
    body_start_index = find_body_start_by_toc(text, toc_items)

    # 본문 시작점 이후의 텍스트만 청크화
    if body_start_index != -1:
        text = text[body_start_index:]

    # 조항을 기준으로 본문을 분리
    chunk_pattern = '|'.join(re.escape(item) for item in toc_items)
    chunks = re.split(f'({chunk_pattern})', text)

    # 조항과 그에 해당하는 본문을 묶어서 반환
    chunked_texts = []
    for i in range(1, len(chunks) - 1, 2):
        chunked_texts.append({
            "Law": law_name,
            "Chapter": "N/A",  # 장 정보가 없는 경우
            "Title": chunks[i].strip(),
            "Body": chunks[i + 1].strip()
        })

    return chunked_texts

## 3. RAG를 위한 전체 법률 청크 및 텍스트파일 저장

In [55]:
import os

# labor_law 디렉토리 안의 PDF 파일 목록 가져오기
pdf_directory = "labor_law"
pdf_files = get_pdf_files(pdf_directory)
# pdf_files = ["채용절차의 공정화에 관한 법률.pdf"]

for file_name in pdf_files:
    file_path = "labor_law/" + file_name
    law_name = file_name[:-4]
    # print(law_name)
    # 본문 pdf -> text 추출
    law_text = extract_text_from_pdf(file_path)
    
    # 장, 조 이름 추출
    chapter = extract_table_of_chapters(law_text)
    toc = extract_table_of_contents(law_text)
    if chapter:
        chunked_text = chunk_by_chapter(law_text, law_name, chapter, toc)
    # 장이 없는 법률
    else:
        chuncked_text = chunk_by_toc(law_text, law_name, toc)
    # 청크 데이터를 파일에 저장
    save_law_chunks_to_text(law_name, chunked_text)

labor_law_texts/산업현장 일학습병행 지원에 관한 법률.txt
labor_law_texts/가사근로자의 고용개선 등에 관한 법률.txt
labor_law_texts/공무원연금법.txt
labor_law_texts/고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률.txt
labor_law_texts/파견근로자 보호 등에 관한 법률.txt
labor_law_texts/구직자 취업촉진 및 생활안정지원에 관한 법률.txt
labor_law_texts/산업재해보상보험법.txt
labor_law_texts/근로기준법.txt
labor_law_texts/임금채권보장법.txt
labor_law_texts/기간제 및 단시간근로자 보호 등에 관한 법률.txt
labor_law_texts/채용절차의 공정화에 관한 법률.txt
labor_law_texts/외국인근로자의 고용 등에 관한 법률.txt
labor_law_texts/장애인고용촉진 및 직업재활법.txt
labor_law_texts/노동위원회법.txt
labor_law_texts/국가인권위원회법.txt
labor_law_texts/어선원 및 어선 재해보상보험법.txt
labor_law_texts/공무원의 노동조합 설립 및 운영 등에 관한 법률.txt
labor_law_texts/중대재해 처벌 등에 관한 법률.txt
labor_law_texts/근로자참여 및 협력증진에 관한 법률.txt
labor_law_texts/고용보험법.txt
labor_law_texts/고용정책 기본법.txt
labor_law_texts/최저임금법.txt
labor_law_texts/산업안전보건법.txt
labor_law_texts/노동조합 및 노동관계조정법.txt
labor_law_texts/근로자퇴직급여 보장법.txt
labor_law_texts/사회적기업 육성법.txt
labor_law_texts/근로복지기본법.txt
labor_law_texts/국민 평생 직업능력 개발법.txt
labor_law_texts/숙련기술장려법.txt


## 4. 텍스트 파일 임베딩
- openai embedding model : https://platform.openai.com/docs/api-reference/embeddings/create?lang=python
- faiss : https://beausty23.tistory.com/203

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 OpenAI API 키 가져오기
project_api_key = os.getenv("PROJECT_API_KEY")
organization_id = os.getenv("ORGANIZATION_ID")
project_id = os.getenv("PROJECT_ID")

# OpenAI API 키 설정
client = OpenAI(
    api_key=project_api_key,
    organization=organization_id,
    project=project_id
)

# 이후 OpenAI API 호출
def generate_embedding(text):
    """
    텍스트 청크를 임베딩으로 변환하는 함수
    """
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002",
        encoding_format="float"
    )
    embedding = response.data[0].embedding
    return embedding

# test
sample_text = "테스트 텍스트"
embedding = generate_embedding(sample_text)
print(embedding)

[-0.016015485, -0.015867319, 0.0021400002, -0.023275658, -0.005795343, 0.026306344, -0.012917452, -0.0025440915, -0.014345241, -0.009011236, -0.012715406, 0.011395374, -0.008903478, -0.0148840295, -0.0045729666, 0.007899985, 0.012883778, -0.009651047, 0.011341495, -0.025834903, 0.018345745, -0.00613882, -0.011738853, 0.020554777, -0.015153424, -0.014143195, 0.016621621, -0.03534452, -0.0010548467, -8.739527e-05, 0.018898003, -0.018884534, -0.007967333, -0.01954455, 0.0026653188, 0.013227255, -0.0065664835, -0.008566735, 0.027774543, -0.017146941, -0.0019985682, 0.007428545, 0.01803594, -0.020635596, 0.00930757, 0.007044658, -0.0023689852, -0.010344737, 0.0044551063, 0.01856126, 0.011449253, 0.003741212, -0.024487933, -0.008654289, 0.0060748393, 0.014910969, -0.018736366, 0.022467475, 0.0070379237, -0.01438565, -0.022467475, 0.0026686862, -0.021605415, 0.009078585, -0.00012143786, -0.00022246067, -0.00019131198, 0.019746594, -0.008890009, 0.014250953, 0.005057876, 0.039250735, 0.0088967

In [ ]:
output = [-0.0025631103, -0.0154960565, 0.000956275, -0.019357027, 0.004813169, 0.018052645, -0.023283217, -0.00073493767, -0.017270016, -0.006270816, -0.018796144, 0.019461378, -0.014622121, -0.020544015, -0.001687544, 0.0016614564, 0.017948294, -0.013109038, 0.006534953, -0.02684418, 0.022174492, -0.008250215, 0.0026902878, 0.018978756, -0.022565806, -0.004963173, 0.013669922, -0.033600878, -0.0058664577, -0.0038772752, 0.009900259, -0.010063306, -0.0063621225, -0.013539484, 0.0016614564, 0.0066034333, 0.0070240963, -0.007506718, 0.0074871522, -0.006691479, 0.0034337854, 0.002510935, 0.011354645, -0.01331774, 0.018978756, 0.015313444, 0.009463291, -0.023204954, 0.0034826996, 0.024678906, 0.018326566, -0.0030114916, -0.030366011, -0.015887372, 0.0037174884, 0.00806108, -0.008321957, 0.017817857, 0.012476413, -0.019148326, -0.013245998, 0.00894806, -0.008719793, 0.0020332052, 0.002223971, 0.0002890021, -0.004242502, 0.01263946, -0.0057262364, 0.013943843, 0.0074675865, 0.03302695, 0.0014511249, -0.007630634, 0.031722568, 0.010500275, -0.0057262364, 0.0017918947, 0.0016777612, 0.00041230695, 0.025148483, -0.011700306, -0.019487465, 0.011178553, 0.0026707218, -0.011367688, -0.005833848, 0.021352733, -0.019643992, -0.014269938, 0.004108803, 0.010722019, -0.0027180058, 0.027757246, -0.001564443, 0.02397454, 0.008589354, 0.036496606, -0.020452708, -0.036287904, -0.00097502547, -0.00225495, -0.011556824, 0.012059011, -0.041557606, -0.006541475, 0.019643992, -0.023061471, 0.020517927, -0.0005604766, -0.010206789, 0.022526676, -0.022631027, -0.03477482, 0.0007398291, 0.029139891, -0.0038446656, -0.034957435, -0.018587442, -0.0035153092, 0.0076958532, 0.01677435, 0.031409517, -0.0022826684, 0.0065806066, -0.0010638864, -0.023061471, -0.040827155, -0.0026185466, -0.009482857, 0.05431446, 0.014830822, 0.020035306, 0.009717645, -0.017322192, -0.0063392962, -0.013298173, 0.009704601, -0.031200815, -0.00884371, 0.018183084, 0.007408889, -0.006919746, -0.034383506, 0.0041381516, 0.01745263, 0.035166137, 0.0011307361, 0.016043898, -0.009241546, -0.0025468057, -0.019630948, 0.034305245, -0.007976295, 0.0025631103, 0.017817857, 0.010604625, -0.0021082072, -0.008530658, -0.022957122, -0.002982143, 0.012319887, 0.019135283, -0.01943529, 0.026557215, 0.038505353, -0.0063164695, 0.01656565, -0.0027277886, -0.0065219095, -0.044766385, 0.030313835, -0.0189918, 0.0067958296, -0.03114864, 0.005354488, 0.015235181, -0.0007178177, -0.005171874, -0.002633221, -0.033366088, 0.011413341, 0.023113647, 0.0050936113, 0.013330783, 0.0001243239, 0.01239815, -0.008419785, -0.009737211, -0.00090083876, 0.03161822, 0.026570259, 0.0061273337, -0.005116438, -0.65156484, -0.0022027749, 0.018248303, -0.0031125813, 0.019683123, 0.03600094, 0.023961496, 0.013435134, -0.01616129, 0.027783334, -0.0033816101, 0.0026185466, -0.020296182, -0.01161552, -0.014152544, -0.029870346, -0.0111329, 0.0014185153, 0.0032837815, 0.02220058, -0.007832813, 0.021104898, -0.0040435838, 0.0077415067, -0.0020886415, 0.023322348, 0.021391863, 0.0037337933, -0.0025843065, 0.024887607, -0.009450247, 0.013043819, 0.006052332, 0.013141648, 0.029948609, 0.0024375636, -0.013539484, -0.029739907, 0.012548154, 0.03114864, -0.01820917, -0.008621965, 0.020309227, 0.0018261346, -0.0021815787, 0.021222293, -0.00087475113, -0.01601781, 0.00577189, 0.008700227, -0.03391393, -0.0009782864, 0.017478717, 0.009613294, -0.010513318, 0.0055012307, 0.026596347, 0.0016337383, -0.004532727, 0.010441577, -0.010258963, 0.007474108, 0.0027995296, -0.010122004, -0.031879093, 0.025083264, -0.013813404, 0.012098142, -0.006711045, -0.038061865, -0.008276303, -0.012026401, 0.0016410755, -0.011172031, 0.022983208, 0.021887528, 0.04505335, -0.011732915, -0.013324261, 0.019383116, -0.0007577644, -0.0102785295, -0.010937242, 0.0016215098, -0.001762546, -0.009880693, -0.04142717, -0.032087795, 0.005569711, -0.0037924903, 0.024913695, 0.00836761, 0.0025565885, -0.008993713, 0.0040696715, 0.020113569, 0.00891545, -0.00083969586, 0.024731081, -0.0022419065, -0.0111133335, 0.0064338637, 0.014022105, 0.012195971, 0.016656958, 0.0022500588, -0.015613452, 0.0023935407, 0.02287886, -0.012430759, 0.0069719213, -0.0022631027, -0.012828596, 0.0007773301, -0.0073175826, -0.027678983, -0.012267712, 0.02114403, 0.001174759, 0.010356792, 0.027705071, -0.0025745237, 0.030131223, 0.01270468, 0.0045131613, -0.00022235635, 0.007832813, -0.019578772, -0.016930876, -0.012359018, -0.030444274, -0.0012734028, 0.021887528, -0.021444038, 0.0091306735, 0.0009978522, 0.01933094, -0.007421933, 0.019643992, -0.023061471, -0.016030854, 0.0066393036, -0.0048033865, -0.00088045775, -0.006332774, -0.020557059, -0.029087717, 0.00075368816, 0.016435212, -0.0009236654, -0.0021815787, 0.011224207, -0.009717645, -0.0018848318, 0.0038414048, -0.00036115074, 0.0061305948, -0.025396315, -0.016317818, 0.0010369836, -0.010180701, 0.015235181, -0.010324183, -0.008080645, -0.0011918789, -0.033653054, -0.05400141, 0.009482857, -0.008576311, -0.044140283, -0.037461847, -0.011693784, -0.00048139846, 0.010715498, 0.00823065, -0.0039946698, 0.0060718977, 0.00080423296, -0.018691793, -0.025735455, -0.0022451673, 0.011798134, 0.006267555, 0.03268781, 0.00022235635, 0.013956887, -0.012587286, 0.012815552, -0.007911076, -0.0030180137, -0.011126378, 0.0047055576, -0.0126785925, 0.0097567765, -0.006212119, -0.0049240417, -0.018104821, 0.019461378, -0.0004151603, 0.019448334, 0.03180083, 0.02424846, 0.024522379, -0.029739907, 0.000113114365, -0.023413654, -0.023348436, -0.005409924, 0.020909242, 0.005605581, 0.007213232, -0.01209162, -0.013311218, 0.0002993964, 0.005687105, 0.013461221, -0.024848476, 0.002947903, -0.016865658, 0.006486039, 0.0013174257, 0.01380036, 0.014635165, -0.026922442, -0.015587363, 0.0060001565, 0.028331175, 0.034618296, 0.025435448, -0.014569946, -0.017348278, -0.0036327036, -0.0056773224, 0.022435369, 0.012313365, -0.00019963157, 0.021717958, -0.019683123, 0.025696324, 0.00922198, -0.0012579133, 0.021326644, 0.016826527, -0.014843866, 0.021417951, 0.021378819, 0.03347044, 0.0075915027, -0.008302391, 0.02927033, -0.025305009, 0.019461378, 0.002042988, -0.014061237, 0.019030932, -0.030026872, -0.0037142274, -0.0015220506, 0.039287984, 0.03665313, 0.012587286, 0.009365462, 0.0059153717, -0.0018766795, 0.021861441, 0.0120524885, 0.001007635, -0.0018848318, -0.006769742, -0.022644069, -0.020961417, -0.0016859137, 0.01755698, -0.015991721, 0.014126456, -0.0007251548, 0.046253383, -0.015222137, 0.014543858, -0.0052925297, -0.0061534215, -0.027861597, 0.0011837266, 0.029583381, 0.0030767107, -0.01772655, -0.026205033, -0.0053773145, -0.0118437875, 0.015326487, -0.0027881162, -0.0014413419, 0.009932868, -0.023609312, -0.010754629, 0.008674139, 0.0071610566, -0.0034240026, 0.012078577, -0.0039294506, 0.017596113, -0.005256659, 0.019670079, -0.021887528, 0.0032903033, -0.00032609547, -0.008361088, -0.018730924, 0.011830743, -0.0077284626, -0.0067045228, 0.01468734, -0.01933094, -0.024326723, -0.001407102, 0.0051033944, 0.0006905072, -0.0033685663, 0.019448334, 0.013017732, -0.0065936507, 0.0018946147, -0.0075523714, 0.0008837187, 0.019304853, -0.012972078, 0.012587286, -0.008041514, -0.00041454888, -0.00026495257, -0.026687654, -0.020752717, -0.00079608057, -0.013330783, 0.0039914087, 0.006619738, 0.013878623, -0.029244242, 0.016109116, 0.021574477, -0.014230806, -0.009261112, 0.0106959315, -7.342244e-05, -0.015378662, -0.00012401819, 0.014948216, -0.0029430117, 0.011289425, -0.011993792, 0.0032968253, 0.011119856, 0.009815474, -0.033992194, -0.019239632, 0.0068023517, -0.008745881, 0.0013728619, -0.009013279, 0.07727158, -0.021522302, 0.0113220345, -0.008198041, 0.0074675865, 0.019500509, 0.014504727, 0.0016826526, 0.002872901, -0.0053447047, -0.0027815944, 0.0075262836, 0.01663087, -0.011295947, -0.0059153717, 0.0067045228, -0.0076175905, -0.02760072, -0.016696088, 0.010337226, 0.010715498, -0.014791691, 0.0138264485, -0.024404986, -0.018926581, -0.009013279, -0.025670236, -0.014856909, -0.013604703, -0.024091933, -0.027026793, 0.0017299365, -0.028905103, -0.034331333, -0.013709053, -0.007995861, -0.009345897, -0.025187615, -0.022865815, 0.01755698, 0.0021636435, 0.012959034, 0.00062202715, -0.025839806, 0.020635322, -0.01977443, -0.0046762093, 0.008002383, -0.0073697576, 0.0068545267, 0.035270486, 0.007721941, 0.017400455, 0.0030652974, 0.0013826448, 0.013187301, 0.006058854, -0.0071871444, -0.022213623, 0.015209093, -0.015352575, -0.0035609626, -0.0091306735, -0.007324104, 0.0065186485, -0.01977443, -0.015313444, 0.0041577173, 0.008811099, 0.009743732, 0.00078425964, -0.010891588, 0.016813483, -0.010558971, 0.0050153485, 0.013435134, -0.0068610488, 0.008152387, -0.0075393273, 0.014987348, 0.0060947244, -0.01616129, 0.0048653446, -0.0032430196, -0.02838335, -0.008941538, -0.020674452, 0.015665626, 0.02623112, 0.0045066397, 0.010676366, -0.02441803, -0.024065847, -0.0008943168, 0.01496126, -0.013604703, -0.002008748, -0.0142438505, -0.013787317, -0.0086089205, -0.01513083, -0.017987426, 0.0187831, 0.0043501137, -0.037044447, -0.0058925455, 0.0022565806, -0.00085844635, -0.0032006272, -0.02838335, -0.024887607, -0.023896277, 0.012972078, -0.011961182, 0.023061471, 0.020478796, 0.0035870501, -0.019200502, -0.0072393194, 0.02650504, -0.025565885, -0.00580776, 0.0023169084, 0.056297123, 0.020361401, 0.02253972, -0.0074349768, 0.0047936034, 0.023309305, -0.011550302, 0.005419707, -0.022070142, 0.003166387, -0.0052142665, 0.0060099396, -0.001622325, 0.0043501137, 0.03349653, 0.0025843065, -0.00017007918, 0.026478952, 0.0034859607, 0.008765446, -0.007813248, -0.008498048, -0.0060132006, 0.0014984087, -0.001420961, -0.010141569, 0.003606616, -0.012548154, 0.011289425, 0.005188179, 0.017217841, 0.021613607, 0.008263259, -0.04106194, -0.010989417, 0.03568789, -0.012385106, -0.018757012, -0.007839335, -0.009769821, -0.02038749, 0.020348357, 0.019565728, 0.009919824, -0.015209093, 0.0020544014, 0.0054392726, 0.0070175747, -0.0151830055, -0.0048653446, 0.009515466, 0.0064632124, -0.03300086, -0.009241546, 0.0027212668, -0.0011780199, 0.012417716, 0.018287433, -0.019578772, 0.009945912, -0.020570103, -0.0037207494, -0.006554519, 0.034931347, 0.028957278, 0.0069719213, 0.028696401, 0.03378349, 0.004907737, -0.021535344, 0.0064338637, -0.021457082, -0.013787317, 0.029661644, 0.020844022, -0.0245876, -0.00584037, -0.0028256173, 0.022774508, -0.003132147, -0.018404828, 0.012085098, 0.009711123, 0.0051294817, -0.0015416164, -0.00093018735, -0.057705857, 0.023152778, 0.0023707142, -0.017413499, -0.021848397, 0.0037142274, -0.01311556, 0.031539954, -0.03667922, 0.020374445, 0.0189918, -0.02182231, -0.021626651, -0.0018587442, -0.035713978, -0.01380036, -0.00015683155, 0.03367914, -0.0030326878, 0.00877849, 0.014752559, 0.0045033786, -0.008882841, 0.013024253, -0.02346583, 0.034826998, -0.0059381984, -0.0047088186, -0.0048620836, 0.017935252, 0.016891746, -0.004454464, 0.008276303, 0.010454621, -0.020896198, -9.9051504e-05, 0.018222215, 0.005569711, 0.00071374147, -0.00090654544, -0.026178945, -0.0066393036, -0.03487917, -0.025839806, 0.0025386533, 0.040566277, -0.015117786, -0.0032756291, -0.011452473, 0.02323104, -0.014739515, -0.029557293, 0.016656958, 0.021222293, -0.014622121, 0.0075197616, -0.0073893233, 0.007363236, 0.0039229286, 0.007891511, 0.009972, -0.007656722, 0.033209562, -0.029739907, -0.019513553, -0.031539954, -0.0189918, -0.0050805677, -0.011602477, -0.021887528, -0.0052827466, 0.0025158266, 0.016135205, 0.017256973, -0.018757012, 0.0089284945, -0.0060066786, 0.0037794465, 0.026087638, -0.026426777, -0.043122865, -6.2263854e-05, -0.013043819, 0.004324026, -0.008556745, -0.02237015, -0.0014715058, 0.016604781, 0.0056479736, -0.022500588, -0.010200267, -0.021861441, -0.015378662, -0.010493753, -0.0084002195, -0.0029381202, -0.013943843, 0.010604625, -0.01567867, 0.018678749, -0.008321957, -0.0106568, -0.020191832, -0.020791847, -0.028565964, -0.023322348, -0.010330705, 0.03454003, -0.0017788508, -0.025474578, -0.036053117, 0.009600251, -0.0521231, -0.0050968723, 0.009841561, -0.0022304931, -0.005895806, 0.0075523714, -0.02066141, 0.024091933, 0.00087556633, 0.027678983, -0.018261347, -0.025318053, 0.009489378, 0.0005780042, -0.019096151, 0.0012913381, -0.012345974, -0.023596268, 0.011126378, 0.015104743, 0.0067045228, 0.042131536, 0.014061237, -0.012822074, 8.646625e-05, 0.0013720467, -0.023244085, -0.0075523714, 0.0141394995, -0.0037272712, -0.0092285015, 0.015235181, 0.01277642, 0.0010068198, -0.0090263225, 0.00877849, -0.0031582348, 0.014935173, -0.012430759, -0.00079934153, 0.004085976, -0.022278843, 0.0033245434, 0.00280116, -0.0038577095, 0.026283296, -8.1371014e-05, -0.008967626, -0.018104821, 0.020544015, -0.0058273263, 0.00935894, 0.00426859, -0.0038805362, -0.009176327, 0.0017234146, 0.013643835, -0.0066132164, -0.008048036, 0.0034729168, -0.01755698, -0.024991957, 0.015613452, 0.023831056, -0.025057176, -0.019839648, -0.027052881, 0.0043598963, -0.002042988, -0.020987505, 0.007630634, -0.019409202, 0.01677435, 0.047453415, -0.008028471, 0.0069719213, -0.004415333, 0.0012750333, -0.006499083, 0.02049184, 0.23374523, -0.0029087716, -0.0042848946, 0.048183866, 0.01386558, 0.039131455, -0.0008388806, -0.021548389, -0.0056414516, 0.018626574, 0.017270016, 0.0010117112, 0.011628564, 0.006055593, -0.013578615, -0.010376358, -0.017987426, -0.0076762876, -0.008791534, -0.043227214, 0.0043533747, -0.008485004, 0.0015986831, 0.003104429, -0.0010386141, -0.023348436, -0.014465596, 0.005957764, 0.012854683, 0.01126986, -0.033418264, -0.01294599, 0.017648287, 0.0026853962, -0.028931191, -0.017217841, 0.032296497, 0.0099654775, 0.045105524, 0.008256737, 0.0012750333, -0.011961182, 0.012209015, -0.0043142433, -0.013539484, 0.026400689, -0.013891667, -0.018587442, -0.027678983, 0.0044348985, -0.013350349, -0.014387332, 0.003375088, 0.011550302, 0.03237476, -0.0034240026, 0.030522536, -0.022044053, 0.008700227, 0.02270929, 0.006120812, 0.013813404, -0.04320113, 0.02992252, -0.02629634, -0.012626417, -0.028070299, 0.019448334, 0.00041414125, -0.010585059, -0.0038511876, -0.0036685742, 0.021026636, 0.0015073763, -0.03897493, -0.030026872, 0.025487622, 1.5374892e-05, 0.026322426, 0.03579224, -0.015887372, 0.008380653, 0.013461221, -0.021874484, -0.02527892, -0.016356949, -0.0013500353, -0.021117942, -0.026818091, -0.010761151, -0.009261112, -0.010128525, 0.0003542212, -0.023544094, 0.0053675314, 0.0178309, -0.0043109823, 0.007708897, 0.0058599357, 0.010193745, -0.009600251, 0.023100603, 0.019865736, 0.027339844, -0.005550145, 0.010924199, -0.014804735, -0.0064273416, 0.007956729, -0.0137220975, -0.0123655405, 0.0004235165, 0.0068871365, -0.0043403306, 0.004457725, 0.014087324, 0.015887372, -0.0077284626, 0.003812056, 0.0017348279, 0.012215536, -0.006841483, -0.013735142, 0.006326252, 0.016696088, -0.011745959, -0.020126613, -0.014191675, 0.015861284, -0.028879015, 0.009078498, -0.009267634, 0.040044524, -0.0022973425, -0.0062219016, -0.005338183, -0.0018261346, 0.01683957, -0.016135205, 0.015522145, 0.02424846, -0.0149743045, -0.013396002, 0.003466395, -0.013604703, -0.04163587, 0.022317974, -0.023165822, -0.018900493, 0.0040142355, -0.028226824, -0.014113412, -0.00059267855, 0.016004765, 0.035635713, -0.007689331, -0.019500509, -0.05058393, 0.005618625, 0.0023185387, -0.037200972, 0.0013288391, 0.048444744, -0.020935329, -0.006958877, -1.1973818e-06, -0.16675217, 0.007415411, -0.0035870501, -0.00058656425, 0.011935094, -0.001840809, 0.051940486, -0.008054558, -0.00090654544, 0.016748264, 0.008745881, -0.02992252, -0.020100525, 0.0135133965, -0.011693784, 0.022696245, -0.030366011, -0.032531284, 0.024222372, 0.011732915, 0.02148317, -0.021117942, 0.01817004, 0.008211084, 0.008895884, -0.0063132085, -0.0060327663, 0.0156004075, 0.009867649, -0.0018457004, -0.0024603903, 0.010878545, 0.012039445, 0.007826291, 0.011713349, 0.005967547, 0.020230964, 0.0029625774, -0.023374524, 0.007630634, 0.006926268, 0.015939547, 0.010637234, -0.008341522, -0.015469969, 0.0064338637, 0.018600486, -0.0034011758, 0.027105056, -0.003531614, 0.00877849, -0.021835353, -0.0071871444, -0.005895806, 0.016930876, 0.015156917, -0.017622199, 0.0026935486, 0.008869797, 0.01800047, 0.0019255938, -0.016252598, 0.005269703, 0.012163362, -0.0049533905, -0.01143943, -0.034357417, 0.018457003, -0.00076754723, 0.002892467, 0.006042549, -0.005178396, 0.008145865, -0.021378819, -0.0005364271, -0.00034423455, -0.03114864, 0.040044524, -0.021352733, 0.002483217, 0.007558893, 0.0070110527, -0.0048653446, 0.016813483, 0.013539484, 0.0146612525, 0.020230964, 0.0027310497, -0.0007928196, -0.0060979854, 0.036627043, -0.034618296, -0.011837265, -0.024783256, 0.01789612, 0.01943529, -0.014230806, 0.0012905229, 0.0037990122, -0.0064925607, -0.011243772, -0.017530892, -0.02916598, 0.026713742, 0.03568789, 0.00013889628, 0.0091306735, 0.0070697498, 0.03678357, 0.004956651, -0.036861833, -0.020139657, 0.012750333, 0.02148317, 0.03367914, 0.022891901, -0.00908502, -0.016317818, 7.840304e-06, 0.017530892, 0.058018908, -0.027339844, -0.027365932, 0.03159213, 0.0063132085, -0.021900572, -0.05045349, -0.03853144, 0.027000705, 0.052879643, 0.002223971, 0.023217998, 0.019135283, 0.016069984, -0.028461613, -0.012378585, -0.0071545346, -0.0050740456, -0.009730689, -0.009482857, 0.00073453004, -0.0035446577, 0.008269781, -0.013578615, -0.0037598808, 0.034826998, -0.008745881, -0.0006118366, 0.015717803, -0.039131455, -0.02418324, 0.0051360037, -0.018874407, 0.038114037, -0.003053884, -0.006140378, -0.005690366, -0.024652818, 0.0036914009, -0.014791691, -0.004206632, -0.00030469545, -0.03829665, -0.010506797, 0.019670079, -0.014439507, 0.009574163, 0.022187537, 0.0069653993, -0.02589198, 0.00055110134, -0.018704837, 0.0074675865, 0.024913695, -0.020165743, -0.03190518, -0.010395924, -0.008419785, -0.04771429, 0.0030049698, 0.030939939, 0.010611147, 0.027965948, 0.001028016, -0.008348044, -0.011608999, -0.022396237, 0.0070045306, -0.00665887, 0.008661096, -0.011328557, 0.01441342, -0.017922208, 0.001482104, 0.005781673, 0.016761307, 0.0059740692, 0.0053447047, -0.023909321, 0.03266172, -0.045235965, 0.002199514, -0.03347044, -0.024548467, 0.012606851, -0.0071023596, -0.025996331, -0.014426464, -0.00556645, -0.011993792, 0.00071129575, 0.01916137, -0.017648287, -0.011759003, 0.0034337854, -0.02872249, 0.004421855, 0.008419785, 0.013656879, -0.023518005, -0.02804421, 0.005233832, 0.011289425, 0.0031305165, 0.028305087, 0.015065611, -0.01496126, -0.036705308, -0.0652191, 0.017700462, 0.013982974, -0.036705308, -0.02059619, 0.01991791, -0.00031142117, -0.008615443, -0.0074545424, 0.0021832092, 0.0050381753, 0.0043794625, 0.00069009955, -0.009795908, -0.024157153, -0.019304853, 0.017413499, 0.0060718977, 0.0089219725, 0.01912224, 0.008380653, -0.023178866, 0.014713428, 0.00584037, -0.015613452, 0.008732837, -0.03821839, 0.0038316217, -0.017778724, -0.0057327584, 0.00925459, -0.029452944, 0.005762107, -0.020478796, -0.0164613, 0.014387332, -0.023478875, 0.021026636, 0.018587442, 0.021391863, -0.01051984, -0.048131693, 0.0063132085, -0.008954582, -0.030157309, 0.02667461, -0.016930876, 0.0027294192, 0.029531207, 0.008380653, 0.0052305716, 0.01082637, -0.013891667, -0.009045889, 0.021248382, -0.0335487, 0.0117394375, 0.0100763505, 0.019291809, -0.018691793, 0.037514023, -0.00024925923, 0.010611147, 0.008993713, 0.0060849413, 0.006329513, -0.0058208043, -6.654386e-05, 0.0034011758, -0.02257885, -0.012652504, 0.00034423455, 0.011661174, 0.021300556, 0.0178309, 0.020557059, 0.004392506, 0.0023723445, -0.006841483, 0.026152857, -0.0126785925, 0.008758925, -0.021848397, -0.0063979933, -0.0009880693, 0.0055925376, -0.006867571, 0.014948216, 0.0044348985, -0.0059773303, -0.055410143, 0.0046142507, 0.008269781, 0.0024245197, 0.0019582033, -0.002636482, 0.029296417, 0.0018538528, 0.020478796, 0.025526755, 0.02561806, 0.0001221839, 0.010969852, -0.01441342, -0.016135205, -0.006632782, -0.0068871365, -0.036496606, 0.0025973504, 0.030287748, 0.007995861, -0.014596034, -0.00847196, 0.0076502, -0.02418324, 0.007134969, -0.019696167, 0.021130987, -0.020713584, 0.021783177, 0.004108803, 0.015039523, 0.04359244, -0.025970243, 0.024313679, 0.014713428, -4.7615034e-05, -0.010878545, 0.0154960565, 0.020648366, 0.006867571, 0.023244085, -0.030026872, -0.022722332, -0.014596034, 0.030783413, -0.004966434, 0.012332931, -0.019552685, 0.067880034, 0.0019745082, -0.019148326, -0.008563267, 9.319197e-05, 0.0043501137, 0.024039758, 0.020817935, -0.016696088, -0.007448021, 0.03454003, -0.007258885, -0.0025565885, -0.04927955, -0.027052881, 0.01943529, -0.019409202, 0.029009454, -0.016643913, 0.021874484, 0.021274468, 0.016696088, -0.0034468293, 0.022722332, -0.009828517, 0.0002806459, 0.027078968, -0.01683957, 0.008204563, -0.020230964, 0.012280756, 0.0010630712, -0.03866188, -0.0110089835, -0.0075915027, -0.004483813, -0.022604939, -0.016343905, 0.009874171, -0.009658948, 0.011289425, 0.03931407, -0.0164613, 0.010865501, 0.0073175826, -0.017139578, -0.016578695, 0.009919824, 0.012952512]

print(len(output))  # 차원 수

In [2]:
# 법률 텍스트 파일이 저장된 디렉토리 경로
text_directory = "labor_law_texts"

# 모든 파일 불러오기
def load_law_texts(directory):
    law_texts = {}
    for file_name in os.listdir(directory):
        if file_name.endswith(".txt"):
            with open(os.path.join(directory, file_name), 'r', encoding='utf-8') as file:
                law_name = file_name[:-4]
                law_texts[law_name] = file.read()
    return law_texts

law_texts = load_law_texts(text_directory)

In [3]:
print(len(law_texts))
print(law_texts.keys())

37
dict_keys(['국가인권위원회법', '어선원 및 어선 재해보상보험법', '노동위원회법', '공무원의 노동조합 설립 및 운영 등에 관한 법률', '중대재해 처벌 등에 관한 법률', '근로자참여 및 협력증진에 관한 법률', '고용보험법', '파견근로자 보호 등에 관한 법률', '산업현장 일학습병행 지원에 관한 법률', '공무원연금법', '가사근로자의 고용개선 등에 관한 법률', '고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률', '산업재해보상보험법', '근로기준법', '기간제 및 단시간근로자 보호 등에 관한 법률', '임금채권보장법', '채용절차의 공정화에 관한 법률', '외국인근로자의 고용 등에 관한 법률', '장애인고용촉진 및 직업재활법', '구직자 취업촉진 및 생활안정지원에 관한 법률', '건설근로자의 고용개선 등에 관한 법률', '교원의 노동조합 설립 및 운영 등에 관한 법률', '직업안정법', '고용상 연령차별금지 및 고령자고용촉진에 관한 법률', '공무원 재해보상법', '남녀고용평등과 일_가정 양립 지원에 관한 법률', '선원법', '산업안전보건법', '노동조합 및 노동관계조정법', '근로자퇴직급여 보장법', '고용정책 기본법', '최저임금법', '근로복지기본법', '숙련기술장려법', '국민 평생 직업능력 개발법', '진폐의 예방과 진폐근로자의 보호 등에 관한 법률', '사회적기업 육성법'])


In [7]:
# test
test_name = "외국인근로자의 고용 등에 관한 법률"
test_text = law_texts["외국인근로자의 고용 등에 관한 법률"]

print(test_name)
print(test_text)

외국인근로자의 고용 등에 관한 법률
========= 외국인근로자의 고용 등에 관한 법률 =========

Chapter: 제1장 총칙
Title: 제1조(목적)
Body: 이 법은 외국인근로자를 체계적으로 도입ㆍ관리함으로써 원활한 인력수급 및 국민경제의 균형 있는 발전
을 도모함을 목적으로 한다.
[전문개정 2009. 10. 9.]

Chapter: 제1장 총칙
Title: 제2조(외국인근로자의 정의)
Body: 이 법에서 “외국인근로자”란 대한민국의 국적을 가지지 아니한 사람으로서 국내에 소재하
고 있는 사업 또는 사업장에서 임금을 목적으로 근로를 제공하고 있거나 제공하려는 사람을 말한다. 다만, 「출입국
관리법」 제18조제1항에 따라 취업활동을 할 수 있는 체류자격을 받은 외국인 중 취업분야 또는 체류기간 등을 고려
하여 대통령령으로 정하는 사람은 제외한다.
[전문개정 2009. 10. 9.]

Chapter: 제1장 총칙
Title: 제3조(적용 범위 등)
Body: ① 이 법은 외국인근로자 및 외국인근로자를 고용하고 있거나 고용하려는 사업 또는 사업장에 적
용한다. 다만, 「선원법」의 적용을 받는 선박에 승무(乘務)하는 선원 중 대한민국 국적을 가지지 아니한 선원 및 그 선
원을 고용하고 있거나 고용하려는 선박의 소유자에 대하여는 적용하지 아니한다.
② 외국인근로자의 입국ㆍ체류 및 출국 등에 관하여 이 법에서 규정하지 아니한 사항은 「출입국관리법」에서 정하
는 바에 따른다.
[전문개정 2009. 10. 9.]

Chapter: 제1장 총칙
Title: 제4조(외국인력정책위원회)
Body: ① 외국인근로자의 고용관리 및 보호에 관한 주요 사항을 심의ㆍ의결하기 위하여 국무총리
소속으로 외국인력정책위원회(이하 “정책위원회”라 한다)를 둔다.
② 정책위원회는 다음 각 호의 사항을 심의ㆍ의결한다.<개정 2021. 4. 13.>
1. 외국인근로자 관련 기본계획의 수립에 관한 사항
2. 외국인근로자 도입 업종 및 규모 등에 관한 사항
3. 외국인근로자를 송출할 

### (에러) 4.1 OpneAI 임베딩 모델에서 허용하는 최대 토큰 수 초과 에러 발생
- text-embedding-ada-002 모델의 경우 최대 8192개의 토큰을 지원하지만, 입력된 텍스트는 10223개의 토큰을 초과

In [10]:
import faiss
import numpy as np
import json

# 각 청크 텍스트를 임베딩 생성
def embed_chunks(law_name, law_text):
    chunks = law_text.split("==================================================")
    embeddings = []
    for chunk in chunks:
        if chunk.strip():
            embedding = generate_embedding(chunk.strip())
            embeddings.append({
                "law": law_name,
                "chunk": chunk.strip(),
                "embedding": embedding
            })
    return embeddings

# FAISS 인덱스 생성
embedding_dimension = 1536
index = faiss.IndexFlatL2(embedding_dimension)

# 법률 청크와 임베딩을 저장
law_embeddings = []

# test
# embeddings = embed_chunks(test_name, test_text)
# 
# # 청크별 임베딩 추가 및 인덱스에 저장
# for embedding_data in embeddings:
#     # 임베딩을 벡터 인덱스에 추가
#     embedding_vector = np.array([embedding_data["embedding"]], dtype=np.float32)
#     index.add(embedding_vector)
# 
#     # law-embeddings 리스트에 저장
#     law_embeddings.append(embedding_data)

# 각 법률 텍스트에 대해 처리
for law_name, law_text in law_texts.items():
    embeddings = embed_chunks(law_name, law_text)

    # 청크별 임베딩 추가 및 인덱스에 저장
    for embedding_data in embeddings:
        # 임베딩을 벡터 인덱스에 추가
        embedding_vector = np.array([embedding_data["embedding"]], dtype=np.float32)
        index.add(embedding_vector)

        # law-embeddings 리스트에 저장
        law_embeddings.append(embedding_data)


# FAISS 인덱스를 파일로 저장
faiss.write_index(index, "law_embeddings.index")
# faiss.write_index(index, "test_embeddings.index")

# 청크 정보와 함께 임베딩을 JSON 파일로 저장
with open("law_embeddings.json", "w", encoding="utf-8") as f:
    json.dump(law_embeddings, f, ensure_ascii=False, indent=4)

print("FAISS 인덱스 및 청크 정보가 성공적으로 저장되었습니다.")
print("total index: ",index.ntotal)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 10223 tokens (10223 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

### 4.2 토큰화 및 청크 나누기
- 텍스트의 길이를 토큰 수 기준으로 나눈 뒤 각 청크가 최대 8192 토큰을 넘지 않도록 설정

In [4]:
import faiss
import numpy as np
import json
import tiktoken

# text-embedding-ada-002 모델을 위한 토크나이저 로드
tokenizer = tiktoken.get_encoding("cl100k_base")

# 각 텍스트를 토큰 길이에 맞춰 나누는 함수 (text-embedding-ada-002의 토크나이저 사용)
def chunk_text_by_tokens(text, max_tokens=8191):    # 안전을 위해 maximum context length -1
    tokens = tokenizer.encode(text)
    chunks = []
    current_chunk = []

    for token in tokens:
        current_chunk.append(token)
        if len(current_chunk) >= max_tokens:
            chunks.append(tokenizer.decode(current_chunk))
            current_chunk = []

    if current_chunk:
        chunks.append(tokenizer.decode(current_chunk))

    return chunks

# 각 조항을 임베딩 생성
def embed_chunks(law_name, law_text):
    # 조 단위로 구분
    clauses = law_text.split("==================================================")
    embeddings = []

    for clause in clauses:
        if clause.strip():
            # 헤더 (Law, Chapter, Title, Body) 분리
            lines = clause.strip().split("\n")
            if len(lines) < 4:
                continue

            # 헤더 정보
            law_header = {
                "Law": law_name,
                "Chapter": lines[0].replace("Chapter:", "").strip(),
                "Title": lines[1].replace("Title:", "").strip()
            }
            # 본문 (Body)만 따로 추출
            body_text = "\n".join(lines[2:])

            # Body를 토크나이저를 사용하여 8191 토큰 이하로 나누기
            smaller_chunks = chunk_text_by_tokens(body_text, max_tokens=8191)

            # 각 청크에 헤더 정보 포함하여 저장
            for chunk in smaller_chunks:
                embedding = generate_embedding(chunk)
                embeddings.append({
                    **law_header,
                    "Body": chunk,
                    "embedding": embedding
                })

    return embeddings

# FAISS 인덱스 생성
embedding_dimension = 1536
index = faiss.IndexFlatL2(embedding_dimension)

# 법률 청크와 임베딩을 저장
law_embeddings = []
cnt = 1
# 각 법률 텍스트에 대해 처리
for law_name, law_text in law_texts.items():
    print(cnt, law_name)
    embeddings = embed_chunks(law_name, law_text)

    # 청크별 임베딩 추가 및 인덱스에 저장
    for embedding_data in embeddings:
        # 임베딩을 벡터 인덱스에 추가
        embedding_vector = np.array([embedding_data["embedding"]], dtype=np.float32)
        index.add(embedding_vector)

        # law-embeddings 리스트에 저장
        law_embeddings.append(embedding_data)
    
    cnt+=1


# FAISS 인덱스를 파일로 저장
faiss.write_index(index, "law_embeddings.index")
# faiss.write_index(index, "test_embeddings.index")

# 청크 정보와 함께 임베딩을 JSON 파일로 저장
with open("law_embeddings.json", "w", encoding="utf-8") as f:
    json.dump(law_embeddings, f, ensure_ascii=False, indent=4)

print("FAISS 인덱스 및 청크 정보가 성공적으로 저장되었습니다.")
print("total index: ",index.ntotal)

/Users/henry/github/ai/venv/include/python3.12/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1 국가인권위원회법
2 어선원 및 어선 재해보상보험법
3 노동위원회법
4 공무원의 노동조합 설립 및 운영 등에 관한 법률
5 중대재해 처벌 등에 관한 법률
6 근로자참여 및 협력증진에 관한 법률
7 고용보험법
8 파견근로자 보호 등에 관한 법률
9 산업현장 일학습병행 지원에 관한 법률
10 공무원연금법
11 가사근로자의 고용개선 등에 관한 법률
12 고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률
13 산업재해보상보험법
14 근로기준법
15 기간제 및 단시간근로자 보호 등에 관한 법률
16 임금채권보장법
17 채용절차의 공정화에 관한 법률
18 외국인근로자의 고용 등에 관한 법률
19 장애인고용촉진 및 직업재활법
20 구직자 취업촉진 및 생활안정지원에 관한 법률
21 건설근로자의 고용개선 등에 관한 법률
22 교원의 노동조합 설립 및 운영 등에 관한 법률
23 직업안정법
24 고용상 연령차별금지 및 고령자고용촉진에 관한 법률
25 공무원 재해보상법
26 남녀고용평등과 일_가정 양립 지원에 관한 법률
27 선원법
28 산업안전보건법
29 노동조합 및 노동관계조정법
30 근로자퇴직급여 보장법
31 고용정책 기본법
32 최저임금법
33 근로복지기본법
34 숙련기술장려법
35 국민 평생 직업능력 개발법
36 진폐의 예방과 진폐근로자의 보호 등에 관한 법률
37 사회적기업 육성법
FAISS 인덱스 및 청크 정보가 성공적으로 저장되었습니다.
total index:  2952


## 5. FAISS 인덱스 로드 및 검색

In [10]:
# 저장된 인덱스 로드
index = faiss.read_index("law_embeddings.index")
# index = faiss.read_index("test_embeddings.index")

# 저장된 청크 정보 로드
with open("law_embeddings.json", "r", encoding="utf-8") as f:
    law_embeddings = json.load(f)

# with open("test_embeddings.json", "r", encoding="utf-8") as f:
#     law_embeddings = json.load(f)

# 질문 임베딩 생성 후 검색 ›
def search_similar_chunks(query_text, top_k=5):
    query_embedding = generate_embedding(query_text)
    query_vector = np.array([query_embedding], dtype=np.float32)
    
    # FAISS 인덱스에서 검색
    distances, indices = index.search(query_vector, top_k)
    print("dist : ",distances)
    print("indices : ", indices)
    # 검색된 인덱스에 해당하는 청크 정보 반환
    results = [law_embeddings[i] for i in indices[0]]
    return results

# 테스트 검색
query = "외국인 취업교육"

search_results = search_similar_chunks(query)
for result in search_results:
    print("Law : ",result["Law"])
    print("Chapter : ",result["Chapter"])
    print("Title : ",result["Title"])
    print("Body : ",result["Body"])

dist :  [[0.21696219 0.2494295  0.26064122 0.26642323 0.27360773]]
indices :  [[1371 1381 2574 1903 1362]]
Law :  외국인근로자의 고용 등에 관한 법률
Chapter :  제2장 외국인근로자 고용절차
Title :  제11조(외국인 취업교육)
Body :  Body: ① 외국인근로자는 입국한 후에 고용노동부령으로 정하는 기간 이내에 한국산업인력공단 또
는 제11조의3에 따른 외국인 취업교육기관에서 국내 취업활동에 필요한 사항을 주지(周知)시키기 위하여 실시하는
교육(이하 “외국인 취업교육”이라 한다)을 받아야 한다. <개정 2010. 6. 4., 2022. 6. 10.>
② 사용자는 외국인근로자가 외국인 취업교육을 받을 수 있도록 하여야 한다.
③ 외국인 취업교육의 시간과 내용, 그 밖에 외국인 취업교육에 필요한 사항은 고용노동부령으로 정한다.<개정
2010. 6. 4.>
[전문개정 2009. 10. 9.]
 
제11조의2(사용자 교육) ① 제8조에 따라 외국인근로자 고용허가를 최초로 받은 사용자는 노동관계법령ㆍ인권 등에 관
한 교육(이하 “사용자 교육”이라 한다)을 받아야 한다.
② 사용자 교육의 내용, 시간, 그 밖에 사용자 교육에 필요한 사항은 고용노동부령으로 정한다.
[본조신설 2021. 4. 13.]
 
제11조의3(외국인 취업교육기관의 지정 등) ① 고용노동부장관은 외국인 취업교육을 전문적ㆍ효율적으로 수행하기 위
하여 외국인 취업교육기관(이하 “외국인 취업교육기관”이라 한다)을 지정할 수 있다.
② 제1항에 따라 외국인 취업교육기관으로 지정을 받으려는 자는 전문인력ㆍ시설 등 대통령령으로 정하는 지정기
준을 갖추어 고용노동부장관에게 신청하여야 한다.
③ 제1항 및 제2항에서 규정한 사항 외에 외국인 취업교육기관의 지정절차 등에 필요한 사항은 고용노동부령으로
정한다.
[본조신설 2022. 6. 10.]
 
제11조의4(외국인 취업교육기관의 지정취소 등) ① 고용노동부